In [5]:
%pip install opencv-python
%pip install cvlib
%pip install scikit-learn
%pip install joblib
%pip install pandas
%pip install numpy
# If not installed already (and for Python 3.11):
%pip install tensorflow==2.20.0

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\sbval\PycharmProjects\MLSN-Team-6\cvenv\Scripts\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\sbval\PycharmProjects\MLSN-Team-6\cvenv\Scripts\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\sbval\PycharmProjects\MLSN-Team-6\cvenv\Scripts\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\sbval\PycharmProjects\MLSN-Team-6\cvenv\Scripts\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\sbval\PycharmProjects\MLSN-Team-6\cvenv\Scripts\python.exe -m pip install --upgrade pip


In [1]:
import cv2
import cvlib as cv
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib
import pandas as pd
import numpy as np
import os

In [7]:

path = r"C:\Users\sbval\PycharmProjects\MLSN-Team-6\tester_images"

import cv2
import numpy as np
from skimage.feature import local_binary_pattern

def load_images_from_folder(base_path):
    data = {'image' : [], 'emotion': []}

    for emotion in os.listdir(base_path):
        emotion_folder = os.path.join(base_path, emotion)

        if not os.path.isdir(emotion_folder):
            continue

        for file in os.listdir(emotion_folder):
            if file.lower().endswith(('.png','.jpg','.jpeg')):
                img_path = os.path.join(emotion_folder,file)
                img = cv2.imread(img_path)

                if img is None:
                    continue

                img = cv2.resize(img, (100, 100))
                data['image'].append(img)
                data['emotion'].append(emotion)

    return pd.DataFrame(data)




In [8]:
train_data = load_images_from_folder(path)


from sklearn.model_selection import train_test_split

X = train_data["image"]
y = train_data["emotion"]


X_train, X_test, Y_train, Y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)
X_train_flat = [img.flatten() for img in X_train]
X_test_flat = [img.flatten() for img in X_test]



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

base_rf = RandomForestClassifier(
    class_weight="balanced",   # keep this because your classes are imbalanced
    random_state=42,
    n_jobs=-1                  # use all cores
)

# Hyperparameter search space
param_distributions = {
    "n_estimators": [100, 200, 300, 400, 500],
    "max_depth": [None, 10, 20, 30, 40, 50],
    "max_features": ["sqrt", "log2", 0.3, 0.5, 0.8],
    "min_samples_split": [2, 4, 6, 8, 10],
    "min_samples_leaf": [1, 2, 3, 4, 5],
    "bootstrap": [True, False]
}

# Randomized search over hyperparameters
rf_search = RandomizedSearchCV(
    estimator=base_rf,
    param_distributions=param_distributions,
    n_iter=40,          # number of random combos to try (can increase if you want)
    scoring="accuracy", # or "f1_macro" if you care about all classes equally
    cv=3,               # 3-fold cross validation
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# IMPORTANT: use your flattened train data here
rf_search.fit(X_train_flat, Y_train)

print("Best hyperparameters found:")
print(rf_search.best_params_)
print(f"Best cross-validated score: {rf_search.best_score_:.4f}")

Fitting 3 folds for each of 40 candidates, totalling 120 fits


In [50]:
model = RandomForestClassifier(
    n_estimators=300,
    max_depth=30,
    max_features="sqrt",
    min_samples_split=4,
    min_samples_leaf=2,
    class_weight="balanced",   # ← balances training
    random_state=42
)

model.fit(X_train_flat, Y_train)



,n_estimators,300
,criterion,'gini'
,max_depth,30
,min_samples_split,4
,min_samples_leaf,2
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [52]:
from sklearn.metrics import accuracy_score

Y_pred = model.predict(X_test_flat)
acc = accuracy_score(Y_test, Y_pred)
print(f"Accuracy: {acc:.4f}")


from sklearn.metrics import classification_report

print(classification_report(Y_test,Y_pred))

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(Y_test,Y_pred)
print(cm)


Accuracy: 0.5951
              precision    recall  f1-score   support

       happy       0.60      0.80      0.69      1433
     neutral       0.56      0.42      0.48       996
         sad       0.51      0.45      0.48       988
    surprise       0.77      0.64      0.70       641

    accuracy                           0.60      4058
   macro avg       0.61      0.58      0.58      4058
weighted avg       0.59      0.60      0.59      4058

[[1147   87  147   52]
 [ 330  415  227   24]
 [ 308  186  445   49]
 [ 126   54   53  408]]


In [53]:
model_filename = 'emotion_detection_model.pkl'
joblib.dump(model, model_filename)





['emotion_detection_model.pkl']